In [1]:
import torch
from rich.markdown import Markdown

device = ('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [2]:
%cd ..

/home/oscarn/flan-gpt2


In [3]:
from models.model_utils import GPT2Model

wrapped_model = GPT2Model("output/gpt2_med_lora-distill_2000-flan_t5_xl_qntz", device, peft=True)
peft_model = wrapped_model.get_model(peft=True)
peft_tokenizer = wrapped_model.get_tokenizer()

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [4]:
from eval.eval_utils import Evaluation
from data.data_utils import create_instruct_dataset


eval = Evaluation(peft_model, peft_tokenizer, device)

2025-09-09 04:59:59,807 - datasets - INFO - PyTorch version 2.5.1 available.


In [5]:
num_samples = 100
CAUSAL_LM = True

In [6]:
template = """
## INPUT PROMPT
{prompt}
## GROUND_TRUTH
{completion}
## MODEL OUTPUT
{prediction}
"""

### ANLI

In [7]:
dataset = create_instruct_dataset(num_samples, ["anli"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], return_full_text=not CAUSAL_LM)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Map:   0%|          | 0/16946 [00:00<?, ? examples/s]

Filter:   0%|          | 0/16946 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating responses... : 100%|██████████| 1/1 [00:01<00:00,  1.02s/it]


INPUT PROMPT                                                    

Determine if the sentence is true based on the text below: Fox Film merged with Nineteenth Century Pictures.       

This is a list of feature films produced by the Fox Film Corporation, including those films produced by its        
corporate predecessor, the Box Office Attractions Company. Some of the later films in this list were produced by   
Fox Film, but were released and distributed by 20th Century Fox after the 1935 merger with Twentieth Century       
Pictures. OPTIONS:                                                                                                 

 • entailment                                                                                                      
 • neutral                                                                                                         
 • contradiction                                                                                                   


                                                   GROUND_TRUTH                                                    

contradiction                                                                                                      


                                                   MODEL OUTPUT                                                    

entailment                                                                                                         

 • contradictionneutral                                                                                            
 • contradictioncontradiction

## Common Gen

In [8]:
dataset = create_instruct_dataset(num_samples, ["common_gen"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], return_full_text=not CAUSAL_LM)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Generating responses... : 100%|██████████| 1/1 [00:00<00:00,  1.35it/s]


INPUT PROMPT                                                    

Write a sentence about the following things:                                                                       

['train', 'station', 'leave']                                                                                      


                                                   GROUND_TRUTH                                                    

a fast train about to leave station                                                                                


                                                   MODEL OUTPUT                                                    

a train leaves a station at night

## SQUAD

In [9]:
dataset = create_instruct_dataset(num_samples, ["squad"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], return_full_text=not CAUSAL_LM)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Using the latest cached version of the dataset since rajpurkar/squad couldn't be found on the Hugging Face Hub
2025-09-09 05:00:23,986 - datasets.load - WARNING - Using the latest cached version of the dataset since rajpurkar/squad couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'plain_text' at /home/oscarn/.cache/huggingface/datasets/rajpurkar___squad/plain_text/0.0.0/7b6d24c440a36b6815f21b70d25016731768db1f (last modified on Fri Nov 22 00:36:36 2024).
2025-09-09 05:00:23,992 - datasets.packaged_modules.cache.cache - WARNING - Found the latest cached dataset configuration 'plain_text' at /home/oscarn/.cache/huggingface/datasets/rajpurkar___squad/plain_text/0.0.0/7b6d24c440a36b6815f21b70d25016731768db1f (last modified on Fri Nov 22 00:36:36 2024).


Filter:   0%|          | 0/87599 [00:00<?, ? examples/s]

Map:   0%|          | 0/87241 [00:00<?, ? examples/s]

Filter:   0%|          | 0/87241 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating responses... : 100%|██████████| 1/1 [00:00<00:00,  1.46it/s]


INPUT PROMPT                                                    

University_of_Notre_Dame This Main Building, and the library collection, was entirely destroyed by a fire in April 
1879, and the school closed immediately and students were sent home. The university founder, Fr. Sorin and the     
president at the time, the Rev. William Corby, immediately planned for the rebuilding of the structure that had    
housed virtually the entire University. Construction was started on the 17th of May and by the incredible zeal of  
administrator and workers the building was completed before the fall semester of 1879. The library collection was  
also rebuilt and stayed housed in the new Main Building for years afterwards. Around the time of the fire, a music 
hall was opened. Eventually becoming known as Washington Hall, it hosted plays and musical acts put on by the      
school. By 1880, a science program was established at the university, and a Science Hall (today LaFortune Student  
Center) was built in 1883. The hall housed multiple classrooms and science labs needed for early research at the   
university.                                                                                                        

Q: Who was the president of Notre Dame in 1879?                                                                    


                                                   GROUND_TRUTH                                                    

Rev. William Corby                                                                                                 


                                                   MODEL OUTPUT                                                    

William Corby                                                                                                      

A: Fr. Sorin

## Cosmos QA

In [12]:
dataset = create_instruct_dataset(num_samples, ["cosmos_qa"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], return_full_text=not CAUSAL_LM)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Using the latest cached version of the module from /home/oscarn/.cache/huggingface/modules/datasets_modules/datasets/allenai--cosmos_qa/3e18538cbfdb2c04189b16642715f0f6da3e97ed5df0aadcec3641245b2cf157 (last modified on Fri Nov 22 00:37:03 2024) since it couldn't be found locally at allenai/cosmos_qa, or remotely on the Hugging Face Hub.
2025-09-09 05:02:06,689 - datasets.load - WARNING - Using the latest cached version of the module from /home/oscarn/.cache/huggingface/modules/datasets_modules/datasets/allenai--cosmos_qa/3e18538cbfdb2c04189b16642715f0f6da3e97ed5df0aadcec3641245b2cf157 (last modified on Fri Nov 22 00:37:03 2024) since it couldn't be found locally at allenai/cosmos_qa, or remotely on the Hugging Face Hub.


Map:   0%|          | 0/25262 [00:00<?, ? examples/s]

Filter:   0%|          | 0/25262 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating responses... : 100%|██████████| 1/1 [00:00<00:00,  1.73it/s]


INPUT PROMPT                                                    

Write a question about the article                                                                                 

I plan on doing some more work next weekend , by which time I 'll have regained my strength . It really is the most
satisfying work ; I used to do chores like this all the time growing up , when I had a huge backyard and vegetable 
garden . Some of the neighbors would let me trim their trees when I asked , and I 'd climb around in them with a   
hatchet and a hand saw , cutting off the branches that were growing too close to windows and power lines . I 'd    
rake leaves and pull weeds for dollars .                                                                           


                                                   GROUND_TRUTH                                                    

What kind of work do they like to do ?                                                                             


                                                   MODEL OUTPUT                                                    

Now will you be more active with the kids?

## CoQA

In [13]:
dataset = create_instruct_dataset(num_samples, ["coqa"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], max_tokens=150, return_full_text=not CAUSAL_LM)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Filter:   0%|          | 0/7199 [00:00<?, ? examples/s]

Map:   0%|          | 0/4063 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4063 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating responses... : 100%|██████████| 1/1 [00:07<00:00,  7.74s/it]


INPUT PROMPT                                                    

Make use of the article to answer the questions.                                                                   

CHAPTER TWELVE.                                                                                                    

SAGE CONVERSE BETWEEN HAKE AND BERTHA--BIARNE IS OUTWITTED--A MONSTER IS SLAIN, AND SAVAGES APPEAR ON THE SCENE.   

Not long after this an event occurred which produced great excitement in the new settlement; namely, the appearance
of natives in the woods. It occurred under the following circumstances.                                            

One morning Karlsefin gave orders for one of the exploring parties to be got ready to go out immediately.          
Karlsefin's plan from the beginning had been to class his men in two divisions. One half stayed at home to work,   
the other half searched the land,--always taking care, however, not to travel so far but that they could return    
home in the evening. They were careful also not to wander far from each other. Sometimes Karlsefin went with the   
exploring party, at other times stayed at home to superintend the work there, while Biarne or Thorward filled his  
place. On the occasion in question Biarne was in charge.                                                           

Soon after the party had started, Hake, who was one of them, observed a female figure disappear round a copse near 
the shores of the lake. At that part they were about to strike off into the thick woods, so Hake went up to Biarne 
and asked leave to go along by the borders of the lake, saying that he could overtake the party again before they  
had reached the Willow Glen, a well-known rendezvous of the hunters and explorers of the colony.                   

"Go as thou wilt, Hake," replied Biarne; "only see to it that ye overtake us before noon, as I intend to go on a   
totally new path to-day."                                                                                          

  1 Who gave orders?                                                                                               
  2 What were they?                                                                                                
  3 Who many groups was he forming?                                                                                
  4 Did the first group stay in the house to play?                                                                 
  5 What did they do?                                                                                              
  6 What of the other?                                                                                             
  7 Did they stay out overnight?                                                                                   
  8 Did the order-giver always attend the searching?                                                               
  9 How many helpers did he have?                                                                                  
 10 Who were they?                                                                                                 
 11 Who saw the woman?                                                                                             
 12 Did she vanish around the house?                                                                               
 13 Where did she vanish?                                                                                          
 14 Was this new happening boring?                                                                                 
 15 What was it?                                                                                                   
 16 Where is the hunter rendevous?                                                                                 
 17 Was this spot secret?                                                                          

## XSum

In [15]:
dataset = create_instruct_dataset(num_samples, ["xsum"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], return_full_text=not CAUSAL_LM)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Using the latest cached version of the dataset since EdinburghNLP/xsum couldn't be found on the Hugging Face Hub
2025-09-09 05:03:17,232 - datasets.load - WARNING - Using the latest cached version of the dataset since EdinburghNLP/xsum couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /home/oscarn/.cache/huggingface/datasets/EdinburghNLP___xsum/default/1.2.0/40db7604fedb616a9d2b0673d11838fa5be8451c (last modified on Tue Sep  9 05:02:59 2025).
2025-09-09 05:03:17,238 - datasets.packaged_modules.cache.cache - WARNING - Found the latest cached dataset configuration 'default' at /home/oscarn/.cache/huggingface/datasets/EdinburghNLP___xsum/default/1.2.0/40db7604fedb616a9d2b0673d11838fa5be8451c (last modified on Tue Sep  9 05:02:59 2025).


Filter:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating responses... : 100%|██████████| 1/1 [00:02<00:00,  2.75s/it]


INPUT PROMPT                                                    

Summarize this article:                                                                                            

Wisconsin Governor Scott Walker declared 13 August "Purple Hearts for Healing" Day for the unnamed girl. In a      
statement, her family thanked him for honouring "our little hero". She was stabbed 19 times by two girls who told  
police it was in "dedication" to Slenderman, a fictional character on the website Creepypasta. The family has been 
raising money for her medical bills and several Wisconsin businesses have come forward to host fundraisers. "The   
compassion and love expressed to our family by the people of Wisconsin [and from people around the world] has      
greatly assisted our family in this healing process," the family said in a statement. "The prayers, cards, hearts, 
notes, financial support, and now this heartfelt proclamation truly demonstrates that through this tragedy that    
there is so much goodness in the world." One of the two accused has been deemed mentally unfit to stand trial.     
Morgan Geyser, 12, and classmate Anissa Weier, 12, have been charged as adults with attempted murder and face up to
60 years in prison. Mr Walker's proclamation includes an invitation to join him in wearing purple in the girl's    
honour. "This little girl is overcoming a terrible ordeal," he said in a statement. "Her strength and determination
are an inspiration, and on behalf of Wisconsin, I wish her well as she continues her journey to recovery."         
According to police, the girls planned to stab the classmate during a sleepover but instead decided to commit the  
crime the next morning in a nearby park. Following their arrest they told investigators about their belief in      
paranormal figure Slenderman and their desire to become his "proxies" by killing to demonstrate their loyalty,     
police said. The victim was found by a cyclist after crawling from the woods with stab wounds to her arms, legs and
torso. Doctors said the knife had just missed a major artery near her heart.                                       


                                                   GROUND_TRUTH                                                    

The family of a 12-year-old allegedly stabbed by two schoolmates have thanked their state governor for honouring   
her with a special day.                                                                                            


                                                   MODEL OUTPUT                                                    

"It is a gruesome and shocking end," the police report said. The girls have been released from the hospital and the
case has been adjourned for further enquiries.                                                                     

Police did not say why the girls did not report the crime.

## BoolQ

In [16]:
dataset = create_instruct_dataset(num_samples, ["bool_q"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], return_full_text=not CAUSAL_LM)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Filter:   0%|          | 0/9427 [00:00<?, ? examples/s]

Map:   0%|          | 0/9399 [00:00<?, ? examples/s]

Filter:   0%|          | 0/9399 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating responses... : 100%|██████████| 1/1 [00:01<00:00,  1.29s/it]


INPUT PROMPT                                                    

Text: The post-Soviet states, also collectively known as the former Soviet Union (FSU) or former Soviet Republics, 
are the states that emerged and re-emerged from the Union of Soviet Socialist Republics in its breakup in 1991,    
with Russia internationally recognised as the successor state to the Soviet Union after the Cold War. The three    
Baltic states were the first to declare their independence, between March and May 1990, claiming continuity from   
the original states that existed prior to their annexation by the Soviet Union in 1940. The remaining 12 republics 
all subsequently seceded. 12 of the 15 states, excluding the Baltic states, initially formed the CIS and most      
joined CSTO, while the Baltic states focused on European Union and NATO membership.                                

Question: the former soviet union broke into separate states?                                                      

OPTIONS:                                                                                                           

 • True                                                                                                            
 • False                                                                                                           


                                                   GROUND_TRUTH                                                    

True                                                                                                               


                                                   MODEL OUTPUT                                                    

True                                                                                                               

 • NeutralAgency                                                                                                   
 • True                                                                                                            
 • FalseTrue                                                                                                       
 • True                                                                                                            
 • None of the above choices.

## Python Code

In [17]:
dataset = create_instruct_dataset(num_samples, ["python_code"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], max_tokens=1000, return_full_text=not CAUSAL_LM)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Filter:   0%|          | 0/16750 [00:00<?, ? examples/s]

Map:   0%|          | 0/16237 [00:00<?, ? examples/s]

Filter:   0%|          | 0/16237 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating responses... : 100%|██████████| 1/1 [00:03<00:00,  3.25s/it]


INPUT PROMPT                                                    

Create a Python program to calculate the average of five numbers entered by a user.                                


                                                   GROUND_TRUTH                                                    

                                                                                                                   
 nums = []                                                                                                         
 for i in range(5):                                                                                                
     nums.append(int(input("Please enter number: ")))                                                              
                                                                                                                   
 #Calculate average                                                                                                
 average = sum(nums) / len(nums)                                                                                   
                                                                                                                   
 #Print average                                                                                                    
 print("The average is ", average)                                                                                 
                                                                                                                   


                                                   MODEL OUTPUT                                                    

                                                                                                                   
 import numpy as np                                                                                                
                                                                                                                   
                                                                                                                   
 r = numpy.random.randint(0, 6)                                                                                    
                                                                                                                   
                                                                                                                   
 for n in range(5):                                                                                                
   r = rnorm(np.pi, r)                                                                                             
                                                                                                                   
 print "The average for 5 numbers is", r                                                                           
                                                                                                                   


## Eng-Spa

In [18]:
dataset = create_instruct_dataset(num_samples, ["eng_spa"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], return_full_text=not CAUSAL_LM)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Generating responses... : 100%|██████████| 1/1 [00:00<00:00,  2.13it/s]


INPUT PROMPT                                                    

Write a sentence in English.                                                                                       


                                                   GROUND_TRUTH                                                    

Tom is definitely up to something.                                                                                 


                                                   MODEL OUTPUT                                                    

What did Tom say to Jane?

## PAWS

In [19]:
dataset = create_instruct_dataset(num_samples, ["paws"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], return_full_text=not CAUSAL_LM)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Map:   0%|          | 0/49401 [00:00<?, ? examples/s]

Filter:   0%|          | 0/49401 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating responses... : 100%|██████████| 1/1 [00:10<00:00, 10.12s/it]


INPUT PROMPT                                                    

Do these sentences have the same meaning? Although five issues of the series were printed , the project was        
finished without any of them being aborted . Although five issues of the series were printed , the project was     
finished without any of them being cancelled .                                                                     

OPTIONS:                                                                                                           

 • No                                                                                                              
 • Yes                                                                                                             


                                                   GROUND_TRUTH                                                    

Yes                                                                                                                


                                                   MODEL OUTPUT                                                    

No                                                                                                                 

 • YesYes                                                                                                          
 • No                                                                                                              
 • YesNo                                                                                                           
 • YesNo                                                                                                           
 • No                                                                                                              
 • YesNo                                                                                                           
 • YesNo                                                                                                           
 • YesNo                                                                                                           
 • None of the above choices.                                                                                      
 • YesNo                                                                                                           
 • None of the above choices.                                                                                      
 • NoNone of the above choices.                                                                                    
 • YesNo                                                                                                           
 • None of the above choices.                                                                                      
 • A few issues of the series were printed, one of them being aborted.                                             
 • Many of the books were printed and completed, a couple of of issues were printed, and a few were cancelled.     
   However, the project was completed without any of the issues being aborted.                                     
 • None of the above choices.                                                                                      
 • The book was printed and distributed in its entirety.It was printed and distributed in its entirety.            

Please tell me if the sentences above mean the same thing. OPTIONS:                                                

 • No                                                                                                              
 • YesYes                                                                                                          
 • YesNo                                                                                                           
 • YesYes                                                                                      

## Quora

In [24]:
dataset = create_instruct_dataset(num_samples, ["quora"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], return_full_text=not CAUSAL_LM)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Filter:   0%|          | 0/56402 [00:00<?, ? examples/s]

Generating responses... : 100%|██████████| 1/1 [00:01<00:00,  1.13s/it]


INPUT PROMPT                                                    

What do you think would happen if a tourist would walk with a Nazi flag in Jerusalem?                              


                                                   GROUND_TRUTH                                                    

Why don’t you give it a try, and then come back and let us know how you got on. We’ll wait.                        


                                                   MODEL OUTPUT                                                    

Say this using Spanish¿El tordo la nazione demotivomentá?

## Alpaca

In [25]:
dataset = create_instruct_dataset(num_samples, ["alpaca"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], return_full_text=not CAUSAL_LM)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Filter:   0%|          | 0/52002 [00:00<?, ? examples/s]

Map:   0%|          | 0/20100 [00:00<?, ? examples/s]

Filter:   0%|          | 0/20100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating responses... : 100%|██████████| 1/1 [00:01<00:00,  1.13s/it]


INPUT PROMPT                                                    

Construct a phrase using the given words Pandemic, challenge, action                                               


                                                   GROUND_TRUTH                                                    

Our action in the face of the pandemic is the ultimate challenge.                                                  


                                                   MODEL OUTPUT                                                    

, and event.A battle is occurring in battle armor.A pandemic is occurring on a planet.